# Object-oriented scientific programming with C++

Matthias Möller, Jonas Thies, Cálin Georgescu, Jingya Li (Numerical Analysis, DIAM)

Lecture 6

## <center>Goal of this lecture</center>

Self-written iterators for user-defined containers

Introduction to the concept of expression templates

Memory management with smart pointers

Latest and greatest features of C++17 and 20

...

## <center>Iterators</center>
<center><img src="plots/lecture5-it0.png"></center>

## <center>Iterators</center>
Fixed size arrays
<center><img src="plots/lecture5-iterator.gif"></center>

## <center>Iterators</center>
<center><img src="plots/lecture5-itend.png"></center>

## <center>Iterators</center>
**Constant iterator** over all entries
```C++
for (auto it = a.cbegin(); it != a.cend(); ++it)
      std::cout << *it << “\n”;
```

**Non-constant iterator** over all entries
```C++
for (auto it = a.begin(); it != a.end(); ++it)
    *it++;
```

## <center>Iterators</center>
<center><img src="plots/lecture7-iterator1.png"></center>

## <center>Iterators</center>
<center><img src="plots/lecture7-iterator2.png"></center>

## <center>Iterators</center>
<center><img src="plots/lecture7-iterator3.png"></center>

## <center>Iterators</center>
<center><img src="plots/lecture7-iterator4.png"></center>

## <center>Iterators</center>
<center><img src="plots/lecture7-iterator5.png"></center>

## <center>Iterators</center>
<center><img src="plots/lecture7-iterator6.png"></center>

## <center>Iterators</center>
<center><img src="plots/lecture7-iterator7.png"></center>

## <center>Iterators</center>
<center><img src="plots/lecture7-iterator8.png"></center>

## <center>Iterators</center>
<center><img src="plots/lecture7-iterator9.png"></center>

## <center>Iterators</center>
<center><img src="plots/lecture7-iterator10.png"></center>

## <center>Iterators</center>
<center><img src="plots/lecture7-iterator11.png"></center>

## <center>Iterators</center>
Create unsorted Vector object and sort it
```C++
Vector<double> v = { 4,2,3,7,5,8,9,1,10,6 };
std::sort(v.begin(), v.end());

std::cout << “v = [“;
for (auto it = v.begin(); it != v.end(); it++)
     std::cout << *it
<< (std::next(it,1) != v.end() ? “,” : “]\n”);
```
Output:

```C++
x = [1,2,3,4,5,6,7,8,9,10]
```

## <center>Our own LA library</center>
**Task:** Let us design our own linear algebra library that supports the following operations:
 - Element-wise Add, Sub, Div, Mul of vectors 
 - Add, Sub, Div, Mul of a vector with a scalar
 - Copy assignment (other functionality can be added later)
 
Additional requirements:
 - Support for arbitrary types: <span style=color:red;>template metaprogramming</span>
 - Mathematical notation: <span style=color:red;>operator overloading</span>

## <center>LA library</center>
**Vector class:** Attributes
```C++
template<typename T> class vector {
private:
     // Attributes
     T* data;
     std::size_t n;
```

## <center>LA library</center>
**Vector class:** Constructors
```C++
template<typename T> class vector {
public:
     // Default constructor
     vector()
     : n(0), data(nullptr) {}
     // Size constructor
     vector(std::size_t n)
     : n(n), data(new T[n]) {}
```

## <center>LA library</center>
**Vector class:** Constructors
```C++
template<typename T>
class vector {
    // Copy constructor
    vector(const vector<T>& other)
    : vector(other.size())
    {
      for (std::size_t i=0; i<other.size(); i++)
          data[i] = other.data[i];
}
```

## <center>LA library</center>
**Vector class:** Constructors
```C++
template<typename T>
class vector {
    // Move constructor
    vector(vector<T>&& other)
    {
      data = other.data; 
      other.data = nullptr;
      n = other.n;       
      other.n    = 0;
    }
```

## <center>LA library</center>
**Vector class:** Destructor and helper functions
```C++
template<typename T>
class vector {
    // Destructor
    ~vector() { delete[] data; data=nullptr; n=0; }
    // Return size
    inline const std::size_t size() const { return n; }
```

## <center>LA library</center>
**Vector class:** Helper functions
```C++
template<typename T>
class vector {
    // Get data pointer (by constant reference)
        inline const T& get(const std::size_t& i) const { 
            return data[i];
    }
    // Get data pointer (by reference)
    inline T& get(const std::size_t& i) {
      return data[i];
    }
```

## <center>LA library</center>
**Vector class:** Assignment operator
```C++
template<typename T>
class vector {
    // Scalar assignment operator
    vector<T>& operator=(const T& value) {
      for (std::size_t i = 0; i < size(); i++)
        data[i] = value;
      return *this;
}
```

## <center>LA library</center>
**Vector class:** Assignment operator
```C++
template<typename T>
class vector {
    // Copy assignment operator
    const vector<T>& operator=(const vector<T>& other) const { 
        if (this != &other) {
        delete[] data; n = other.size(); data = new T[n];
        for (std::size_t i = 0; i < other.size(); i++)
              data[i] = other.data[i];
  }
  return *this;
}
```

## <center>LA library</center>
**Vector class:** Assignment operator
```C++
template<typename T>
class vector {
    // Move assignment operator
    vector<T>& operator=(vector<T>&& other) { 
        if (this != &other) {
            std::swap(this->data, other.data);
            std::swap(this->n, other.n);
            other.n = 0; delete[] other.data; other.data = nullptr;
  }
  return *this;
}
```

## <center>LA library</center>
**Vector class:** Binary vector-vector operators **(outside class!)**
```C++
template<typename T1, typename T2>
auto operator+(const vector<T1>& v1,
               const vector<T2>& v2) { 
    vector<typename std::common_type<T1,T2>::type>
      v(v1.size());
    for (std::size_t i=0; i<v1.size(); i++)
      v.get[i] = v1.get[i] + v2.get[i];
    return v;
}
```
Similar for `operator-`, `operator*` and `operator/`

## <center>LA library</center>
**Vector class:** Binary vector-scalar operators **(outside class!)**
```C++
template<typename T1, typename T2>
auto operator+(const vector<T1>& v1,
               const T2& s2) { 
    vector<typename std::common_type<T1,T2>::type>
      v(v1.size());
    for (std::size_t i=0; i<v1.size(); i++)
      v.get[i] = v1.get[i] + s2;
    return v;
}
```
Similar for `operator-`, `operator*` and `operator/`

## <center>Quiz: LA library</center>
What is the **theoretical complexity** (#memory transfers and #floating-point operations) of the expression `z=2*x+y/(x-3)`?

## <center>Quiz: LA library</center>
What is the **theoretical complexity** (#memory transfers and #floating-point operations) of the expression `z=2*x+y/(x-3)`?

- <span style=color:red;>Two loads (x,y) and one store (z) each of length n</span>
- <span style=color:red;>Four times n floating-point operations (*,+,/,-)</span>

## <center>Quiz: LA library</center>
What is the **theoretical complexity** (#memory transfers and #floating-point operations) of the expression `z=2*x+y/(x-3)`?

- <span style=color:red;>Two loads (x,y) and one store (z) each of length n</span>
- <span style=color:red;>Four times n floating-point operations (*,+,/,-)</span>

What is the **practical complexity** of the following code?
```C++
int main() {
    Vector<double> x(10), y(10), z(10); x=1.0; y=2.0; z=2*x+y/(x-3);
}
```

## <center>Quiz: LA library</center>
What is the **theoretical complexity** (#memory transfers and #floating-point operations) of the expression `z=2*x+y/(x-3)`?

- <span style=color:red;>Two loads (x,y) and one store (z) each of length n</span>
- <span style=color:red;>Four times n floating-point operations (*,+,/,-)</span>

What is the **practical complexity** of the following code?
```C++
int main() {
    Vector<double> x(10), y(10), z(10); x=1.0; y=2.0; z=2*x+y/(x-3);
}
```
- <span style=color:red;>Five (!) loads and three (!) stores each of length n</span>

## <center>LA expression template library</center>

<center><img src="plots/lecture7-LAexpression.png"></center>

## <center>LA expression template library</center>
Implement templated vector class (derived from base class `vectorBase`) as before but remove all operators (+,-,*,/)

Implement an additional **assignment operator**, which loops through the <span style=color:red;>expression e</span> and assigns the value to `data[i]`
```C++
template <typename E>
  vector<T>& operator=(const E& e) {
      for (std::size_t i = 0; i < size(); i++)
          data[i] = e.get(i);
      return *this;
  }
```

## <center>LA expression template library</center>
For each **binary operator** we need to implement a <span style=color:red;>helper class</span> that represents the operation in the expression tree
```C++
template<typename E1, typename E2>
class VectorAdd : vectorBase {
private:
    const E1& e1;
    const E2& e2;
    
public:
    VectorAdd(const E1& e1, const E2& e2)
    : e1(e1), e2(e2)
{}
```